In [1]:
from core.db import get_engine
from core.libs import pd, text, np

In [10]:
query1 = text("""
    SELECT COUNT(*) AS null_survival_imc
    FROM masterdatabase.inventory_metrics_current imc
    JOIN masterdatabase.contract_tree_information cti
        ON cti.contract_code = imc.contract_code
    WHERE imc.survival IS NULL
      AND COALESCE(cti.status, '') <> 'Out of Program'
""")


query2 = text("""

    SELECT
        imc.contract_code,
        imc.inventory_year AS imc_year,
        im.inventory_year AS im_year,
        im.survival AS im_survival
    FROM masterdatabase.inventory_metrics_current imc
    JOIN masterdatabase.contract_tree_information cti
        ON cti.contract_code = imc.contract_code
    LEFT JOIN masterdatabase.inventory_metrics im
        ON im.contract_code = imc.contract_code
    WHERE imc.survival IS NULL
      AND im.contract_code IS NOT NULL
      AND COALESCE(cti.status, '') <> 'Out of Program';


""")

In [11]:
query3=text("""
    SELECT
        imc.contract_code,
        cti.etp_year,
        cti.planting_year,
        cti.region,
        cti.status,
        imc.inventory_year AS imc_year,
        im.inventory_year AS im_year,
        im.survival,
        im.rel_path
    FROM masterdatabase.inventory_metrics_current imc
    JOIN masterdatabase.contract_tree_information cti
        ON cti.contract_code = imc.contract_code
    LEFT JOIN masterdatabase.inventory_metrics im
        ON im.contract_code = imc.contract_code
    WHERE imc.survival IS NULL
      AND im.contract_code IS NULL
      AND COALESCE(cti.status, '') <> 'Out of Program'
    ORDER BY cti.etp_year, cti.region, imc.contract_code;
""")

In [12]:
query4=text("""
    SELECT
        cti.contract_code AS cti_code,
        imc.contract_code AS imc_code,
        im.contract_code AS im_code,
        cti.etp_year,
        im.inventory_year AS im_year,
        im.survival,
        im.doyle_bf_total,
        im.rel_path,
        imc.inventory_year AS imc_year,
        COUNT(*) OVER (PARTITION BY im.contract_code) AS im_count
    FROM masterdatabase.inventory_metrics_current imc
    JOIN masterdatabase.contract_tree_information cti
        ON cti.contract_code = imc.contract_code
    LEFT JOIN masterdatabase.inventory_metrics im
        ON im.contract_code = imc.contract_code
    WHERE imc.survival IS NULL
      AND COALESCE(cti.status, '') <> 'Out of Program'
    ORDER BY cti.contract_code, im.inventory_year;
""")

In [13]:
engine = get_engine()

queries = [query1, query2, query3, query4]

for idx, q in enumerate(queries, start=1):
    df = pd.read_sql_query(q, engine)
    varname = f"df_query{idx}"
    globals()[varname] = df


💻 Conectado a la base de datos helloworldtree


In [14]:
print(df_query1.head())
print(df_query2.head())

   null_survival_imc
0                 52
  contract_code imc_year  im_year  im_survival
0        MX0005     None      NaN          NaN
1        CR0037     None      NaN          NaN
2        CR0037     None   2024.0        49.03
3        CR0037     None   2024.0        49.03
4        CR0037     None   2024.0        49.03


In [15]:
print(len(df_query4))
print(df_query4.head())


140
  cti_code imc_code im_code  etp_year  im_year  survival  doyle_bf_total  \
0   CR0037   CR0037  CR0037      2020   2024.0     49.03             0.0   
1   CR0037   CR0037  CR0037      2020   2024.0     49.03             0.0   
2   CR0037   CR0037  CR0037      2020   2024.0     49.03             0.0   
3   CR0037   CR0037  CR0037      2020   2024.0     49.03             0.0   
4   CR0037   CR0037  CR0037      2020      NaN       NaN             NaN   

  rel_path imc_year  im_count  
0     None     None         5  
1     None     None         5  
2     None     None         5  
3     None     None         5  
4     None     None         5  


In [16]:
engine = get_engine()

def print_columns(table):
    q = f"SELECT * FROM {table} LIMIT 0"
    df = pd.read_sql_query(q, engine)
    print(f"\n📌 Columnas de {table}:")
    print(list(df.columns))

tables = ["masterdatabase.inventory_metrics_current",
         "masterdatabase.inventory_metrics"
        ]

# tables = [
#     "masterdatabase.contract_tree_information",
#     "masterdatabase.inventory_metrics_current",
#     "masterdatabase.inventory_metrics"
# ]

for t in tables:
    print_columns(t)


💻 Conectado a la base de datos helloworldtree

📌 Columnas de masterdatabase.inventory_metrics_current:
['rel_path', 'contract_code', 'planting_year', 'tree_age', 'inventory_year', 'inventory_date', 'tht_mean', 'tht_std', 'mht_mean', 'mht_std', 'dbh_mean', 'dbh_std', 'noncom_dbh_count', 'doyle_bf_mean', 'doyle_bf_std', 'doyle_bf_total', 'pkid', 'progress', 'total_trees', 'planting_date', 'type_of_metric', 'rn', 'survival', 'mht_pct_of_target', 'dbh_pct_of_target', 'projected_dbh', 'projected_doyle_bf', 'mortality']

📌 Columnas de masterdatabase.inventory_metrics:
['rel_path', 'contract_code', 'tree_age', 'inventory_year', 'inventory_date', 'tht_mean', 'tht_std', 'mht_mean', 'mht_std', 'dbh_mean', 'dbh_std', 'noncom_dbh_count', 'doyle_bf_mean', 'doyle_bf_std', 'doyle_bf_total', 'pkid', 'progress', 'total_trees', 'type_of_metric', 'planting_date', 'planting_year', 'survival', 'mht_pct_of_target', 'dbh_pct_of_target', 'projected_dbh', 'projected_doyle_bf', 'mortality']


In [17]:
from core.db import get_engine
from core.libs import pd, text

engine = get_engine()

# ============================================================================
# QUERY 1: Panorama general de IM
# ¿Cuántas filas tiene IM? ¿Cuántas tienen survival NULL?
# ============================================================================
q1_overview = text("""
    SELECT
        COUNT(*) as total_rows,
        COUNT(survival) as rows_with_survival,
        COUNT(*) - COUNT(survival) as rows_null_survival,
        COUNT(DISTINCT contract_code) as unique_contracts,
        COUNT(DISTINCT inventory_year) as unique_years,
        COUNT(DISTINCT rel_path) as unique_paths
    FROM masterdatabase.inventory_metrics;
""")

# ============================================================================
# QUERY 2: Filas con survival NULL
# ¿Cuántas filas tienen survival NULL?
# ============================================================================
q2_empty_rows = text("""
    SELECT
        COUNT(*) as rows_with_null_survival
    FROM masterdatabase.inventory_metrics
    WHERE survival IS NULL;
""")

# ============================================================================
# QUERY 3: Detalle de los 119 contratos problemáticos
# ¿Qué tienen estas filas además del contract_code?
# ============================================================================
q3_ghost_details = text("""
    SELECT
        im.contract_code,
        im.inventory_year,
        im.survival,
        im.dbh_mean,
        im.mht_mean,
        im.doyle_bf_total,
        im.rel_path,
        cti.etp_year,
        cti.planting_year,
        cti.region,
        cti.status
    FROM masterdatabase.inventory_metrics im
    JOIN masterdatabase.contract_tree_information cti
        ON cti.contract_code = im.contract_code
    WHERE im.contract_code IN (
        SELECT imc.contract_code
        FROM masterdatabase.inventory_metrics_current imc
        JOIN masterdatabase.contract_tree_information cti2
            ON cti2.contract_code = imc.contract_code
        WHERE imc.survival IS NULL
          AND COALESCE(cti2.status, '') <> 'Out of Program'
    )
    ORDER BY cti.etp_year, cti.region, im.contract_code;
""")

# ============================================================================
# QUERY 4: ¿Existen múltiples filas para estos contratos?
# ¿Los 119 tienen solo 1 fila vacía, o tienen múltiples?
# ============================================================================
q4_row_count = text("""
    SELECT
        im.contract_code,
        COUNT(*) as row_count,
        COUNT(survival) as rows_with_data,
        MAX(im.inventory_year) as latest_year
    FROM masterdatabase.inventory_metrics im
    WHERE im.contract_code IN (
        SELECT imc.contract_code
        FROM masterdatabase.inventory_metrics_current imc
        JOIN masterdatabase.contract_tree_information cti
            ON cti.contract_code = imc.contract_code
        WHERE imc.survival IS NULL
          AND COALESCE(cti.status, '') <> 'Out of Program'
    )
    GROUP BY im.contract_code
    ORDER BY row_count DESC, im.contract_code;
""")

# ============================================================================
# QUERY 5: Distribución por inventory_year
# ¿Cuándo se registraron estos inventarios? ¿Son recientes o antiguos?
# ============================================================================
q5_timestamps = text("""
    SELECT
        im.inventory_year,
        COUNT(*) as row_count,
        COUNT(DISTINCT contract_code) as contracts_affected,
        COUNT(survival) as rows_with_survival
    FROM masterdatabase.inventory_metrics im
    WHERE contract_code IN (
        SELECT imc.contract_code
        FROM masterdatabase.inventory_metrics_current imc
        JOIN masterdatabase.contract_tree_information cti
            ON cti.contract_code = imc.contract_code
        WHERE imc.survival IS NULL
          AND COALESCE(cti.status, '') <> 'Out of Program'
    )
    GROUP BY im.inventory_year
    ORDER BY im.inventory_year DESC;
""")

# ============================================================================
# QUERY 6: ¿Existen variantes de contract_code en IM?
# ¿Los códigos tienen sufijos que no matchean con CTI?
# ============================================================================
q6_code_variants = text("""
    SELECT
        im.contract_code as im_code,
        cti.contract_code as cti_code,
        im.survival,
        im.inventory_year,
        im.rel_path
    FROM masterdatabase.inventory_metrics im
    LEFT JOIN masterdatabase.contract_tree_information cti
        ON SUBSTRING(im.contract_code, 1, 6) = SUBSTRING(cti.contract_code, 1, 6)
    WHERE im.contract_code LIKE 'MX%'
       OR im.contract_code LIKE 'CR%'
       OR im.contract_code LIKE 'US%'
       AND im.survival IS NULL
    LIMIT 100;
""")

# ============================================================================
# Ejecutar todas las queries
# ============================================================================
print("=" * 80)
print("🔍 AUDITORÍA PROFUNDA DE INVENTORY_METRICS (IM)")
print("=" * 80)

print("\n📊 QUERY 1: Panorama General de IM")
print("-" * 80)
df1 = pd.read_sql_query(q1_overview, engine)
print(df1.to_string(index=False))

print("\n\n🕳️  QUERY 2: Filas con Survival NULL")
print("-" * 80)
df2 = pd.read_sql_query(q2_empty_rows, engine)
print(df2.to_string(index=False))

print("\n\n👻 QUERY 3: Detalle de los 119 Contratos Problemáticos (primeros 20)")
print("-" * 80)
df3 = pd.read_sql_query(q3_ghost_details, engine)
print(df3.head(20).to_string(index=False))
print(f"\n... Total: {len(df3)} filas")

print("\n\n🔢 QUERY 4: Conteo de Filas por Contrato")
print("-" * 80)
df4 = pd.read_sql_query(q4_row_count, engine)
print(df4.head(20).to_string(index=False))
print(f"\n... Total: {len(df4)} contratos")

print("\n\n⏰ QUERY 5: Distribución por Inventory Year")
print("-" * 80)
df5 = pd.read_sql_query(q5_timestamps, engine)
print(df5.to_string(index=False))

print("\n\n🔤 QUERY 6: Variantes de Contract Code (primeros 20)")
print("-" * 80)
df6 = pd.read_sql_query(q6_code_variants, engine)
print(df6.head(20).to_string(index=False))

# ============================================================================
# Guardar resultados en variables globales para análisis posterior
# ============================================================================
globals()['df_im_overview'] = df1
globals()['df_im_empty'] = df2
globals()['df_im_ghosts'] = df3
globals()['df_im_counts'] = df4
globals()['df_im_timestamps'] = df5
globals()['df_im_variants'] = df6

print("\n\n✅ Auditoría completada. DataFrames disponibles:")
print("   - df_im_overview")
print("   - df_im_empty")
print("   - df_im_ghosts")
print("   - df_im_counts")
print("   - df_im_timestamps")
print("   - df_im_variants")

💻 Conectado a la base de datos helloworldtree
🔍 AUDITORÍA PROFUNDA DE INVENTORY_METRICS (IM)

📊 QUERY 1: Panorama General de IM
--------------------------------------------------------------------------------
 total_rows  rows_with_survival  rows_null_survival  unique_contracts  unique_years  unique_paths
        512                 286                 226               424             4             0


🕳️  QUERY 2: Filas con Survival NULL
--------------------------------------------------------------------------------
 rows_with_null_survival
                     226


👻 QUERY 3: Detalle de los 119 Contratos Problemáticos (primeros 20)
--------------------------------------------------------------------------------
contract_code  inventory_year  survival dbh_mean mht_mean  doyle_bf_total rel_path  etp_year  planting_year     region status
       US0001             NaN       NaN     None     None             NaN     None      2015           2015        USA    RFT
       US0037         

In [18]:
from core.db import get_engine
from core.libs import pd, text

engine = get_engine()

# ============================================================================
# Obtener la lista de los 119 contratos problemáticos
# ============================================================================
missing_contracts_query = text("""
    SELECT DISTINCT imc.contract_code
    FROM masterdatabase.inventory_metrics_current imc
    JOIN masterdatabase.contract_tree_information cti
        ON cti.contract_code = imc.contract_code
    WHERE imc.survival IS NULL
      AND COALESCE(cti.status, '') <> 'Out of Program'
    ORDER BY imc.contract_code;
""")

df_missing = pd.read_sql_query(missing_contracts_query, engine)
missing_contracts = tuple(df_missing['contract_code'].tolist())

print("=" * 80)
print(f"🔍 BUSCANDO {len(missing_contracts)} CONTRATOS EN TABLAS SOURCE")
print("=" * 80)
print("\nContratos a buscar:")
print(df_missing.to_string(index=False))

# ============================================================================
# Obtener todas las tablas inventory_* del schema PUBLIC
# ============================================================================
tables_query = text("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
      AND table_name LIKE 'inventory_%'
    ORDER BY table_name;
""")

df_tables = pd.read_sql_query(tables_query, engine)
inventory_tables = df_tables['table_name'].tolist()

print(f"\n📋 Tablas inventory_* encontradas en schema PUBLIC: {len(inventory_tables)}")
for t in inventory_tables:
    print(f"   - {t}")

# ============================================================================
# Buscar los contratos faltantes en cada tabla source
# ============================================================================
print("\n" + "=" * 80)
print("🔎 BUSCANDO CONTRATOS FALTANTES EN CADA TABLA SOURCE")
print("=" * 80)

found_any = False
results = []

for table_name in inventory_tables:
    # Query para buscar en esta tabla específica
    search_query = text(f"""
        SELECT DISTINCT contractcode, '{table_name}' as source_table
        FROM public.{table_name}
        WHERE contractcode = ANY(:codes)
    """)

    try:
        df = pd.read_sql_query(
            search_query,
            engine,
            params={'codes': list(missing_contracts)}
        )

        if not df.empty:
            found_any = True
            results.append(df)
            print(f"\n⚠️  ENCONTRADOS en {table_name}: {len(df)} contratos")
            print(df.to_string(index=False))

    except Exception as e:
        print(f"\n⚠️  Error al consultar {table_name}: {e}")

# ============================================================================
# Resumen final
# ============================================================================
print("\n" + "=" * 80)
print("📊 RESUMEN")
print("=" * 80)

if found_any:
    df_all_found = pd.concat(results, ignore_index=True)
    print(f"\n✅ Se encontraron {len(df_all_found)} coincidencias en tablas source")
    print("\nContratos encontrados por tabla:")
    print(df_all_found.groupby('source_table').size().to_string())

    # ¿Cuántos de los 119 SÍ están en source?
    found_codes = set(df_all_found['contractcode'].unique())
    missing_codes = set(missing_contracts)
    truly_missing = missing_codes - found_codes

    print(f"\n📌 De los {len(missing_contracts)} contratos problemáticos:")
    print(f"   - {len(found_codes)} SÍ existen en tablas source")
    print(f"   - {len(truly_missing)} NO existen en ninguna tabla source")

    if truly_missing:
        print("\n🚨 Contratos que NO están en ninguna tabla source:")
        for code in sorted(truly_missing):
            print(f"   - {code}")
else:
    print("\n✅ NINGUNO de los 119 contratos está presente en tablas raw de inventario.")
    print("   → Esto significa que nunca se ingested ningún archivo Cruise para estos contratos")
    print("   → Las filas vacías en IM fueron creadas como placeholders")

# Guardar resultados
if found_any:
    globals()['df_found_in_source'] = df_all_found
    print("\n✅ DataFrame disponible: df_found_in_source")

💻 Conectado a la base de datos helloworldtree
🔍 BUSCANDO 52 CONTRATOS EN TABLAS SOURCE

Contratos a buscar:
contract_code
       CR0037
       CR0040
       CR0045
       CR0072
       CR0099
      GT0024a
       MX0005
       MX0027
       MX0028
       MX0030
       MX0033
       MX0034
       MX0036
       MX0037
       MX0041
       MX0045
       US0001
       US0037
       US0038
       US0041
       US0043
       US0045
       US0049
       US0052
       US0054
       US0058
       US0061
       US0064
       US0069
       US0074
       US0076
       US0079
       US0080
       US0084
       US0085
       US0090
       US0094
       US0110
       US0112
       US0116
       US0129
       US0130
       US0132
       US0133
       US0134
       US0135
       US0136
       US0138
       US0140
       US0142
       US0143
       US0144

📋 Tablas inventory_* encontradas en schema PUBLIC: 16
   - inventory_cr_2021
   - inventory_cr_2022
   - inventory_cr_2024
   - inventory_cr_2025
   

In [19]:
from core.db import get_engine
from core.libs import pd, text

engine = get_engine()

# ============================================================================
# PASO 1: Obtener los 119 contratos problemáticos
# ============================================================================
missing_contracts_query = text("""
    SELECT DISTINCT imc.contract_code
    FROM masterdatabase.inventory_metrics_current imc
    JOIN masterdatabase.contract_tree_information cti
        ON cti.contract_code = imc.contract_code
    WHERE imc.survival IS NULL
      AND COALESCE(cti.status, '') <> 'Out of Program'
    ORDER BY imc.contract_code;
""")

df_119 = pd.read_sql_query(missing_contracts_query, engine)
codes_119 = set(df_119['contract_code'].tolist())

# ============================================================================
# PASO 2: Identificar cuáles de los 119 SÍ están en tablas source
# ============================================================================
# Asumiendo que ya ejecutaste el script anterior y tienes df_found_in_source
# Si no, necesitas ejecutar primero el script de búsqueda en source

try:
    codes_found_in_source = set(df_found_in_source['contractcode'].unique())
except NameError:
    print("⚠️  df_found_in_source no está disponible.")
    print("Por favor ejecuta primero el script 'search_source_inventories'\n")

    # Alternativa: buscar directamente en las tablas
    print("Buscando en tablas source...")

    tables_query = text("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
          AND table_name LIKE 'inventory_%'
        ORDER BY table_name;
    """)

    df_tables = pd.read_sql_query(tables_query, engine)
    inventory_tables = df_tables['table_name'].tolist()

    codes_found_in_source = set()

    for table_name in inventory_tables:
        search_query = text(f"""
            SELECT DISTINCT contractcode
            FROM public.{table_name}
            WHERE contractcode = ANY(:codes)
        """)

        try:
            df = pd.read_sql_query(
                search_query,
                engine,
                params={'codes': list(codes_119)}
            )
            codes_found_in_source.update(df['contractcode'].tolist())
        except:
            continue

# ============================================================================
# PASO 3: Los 86 que NO están en source
# ============================================================================
codes_86_not_in_source = codes_119 - codes_found_in_source

print("=" * 80)
print(f"🔍 ANÁLISIS DE LOS {len(codes_86_not_in_source)} CONTRATOS SIN DATOS SOURCE")
print("=" * 80)

# ============================================================================
# PASO 4: Consultar CTI para ver su status
# ============================================================================
status_query = text("""
    SELECT
        contract_code,
        etp_year,
        planting_year,
        region,
        status,
        COALESCE(status, 'NULL') as status_clean
    FROM masterdatabase.contract_tree_information
    WHERE contract_code = ANY(:codes)
    ORDER BY
        CASE
            WHEN status = 'Out of Program' THEN 1
            WHEN status IS NULL THEN 2
            ELSE 3
        END,
        etp_year, region, contract_code;
""")

df_86_status = pd.read_sql_query(
    status_query,
    engine,
    params={'codes': list(codes_86_not_in_source)}
)

# ============================================================================
# PASO 5: Análisis por status
# ============================================================================
print(f"\n📊 Distribución por STATUS:")
print("-" * 80)
status_counts = df_86_status.groupby('status_clean').size().sort_values(ascending=False)
print(status_counts.to_string())

print(f"\n📊 Distribución por REGION:")
print("-" * 80)
region_counts = df_86_status.groupby('region').size().sort_values(ascending=False)
print(region_counts.to_string())

print(f"\n📊 Distribución por ETP_YEAR:")
print("-" * 80)
year_counts = df_86_status.groupby('etp_year').size().sort_values(ascending=False)
print(year_counts.to_string())

# ============================================================================
# PASO 6: Separar por status
# ============================================================================
df_out_of_program = df_86_status[df_86_status['status'] == 'Out of Program']
df_not_out_of_program = df_86_status[df_86_status['status'] != 'Out of Program']

print("\n" + "=" * 80)
print("🚨 HALLAZGOS CRÍTICOS")
print("=" * 80)

print(f"\n De los {len(codes_86_not_in_source)} contratos sin datos source:")
print(f"   ✅ {len(df_out_of_program)} son 'Out of Program' (ESPERADO)")
print(f"   ⚠️  {len(df_not_out_of_program)} NO son 'Out of Program' (PROBLEMÁTICO)")

if len(df_out_of_program) > 0:
    print(f"\n📋 Contratos 'Out of Program' (primeros 20):")
    print("-" * 80)
    print(df_out_of_program.head(20).to_string(index=False))

if len(df_not_out_of_program) > 0:
    print(f"\n🚨 Contratos SIN datos source que NO son 'Out of Program':")
    print("-" * 80)
    print(df_not_out_of_program.to_string(index=False))

    print(f"\n❗ Estos {len(df_not_out_of_program)} contratos son PROBLEMÁTICOS porque:")
    print("   1. Están marcados como activos (o RFT, etc.)")
    print("   2. NO tienen datos en ninguna tabla inventory_*")
    print("   3. Tienen survival = NULL en IMC")
    print("   4. Deberían tener un inventario pero no lo tienen")

# ============================================================================
# PASO 7: Los 33 que SÍ están en source
# ============================================================================
codes_33_in_source = codes_119 & codes_found_in_source

status_33_query = text("""
    SELECT
        contract_code,
        etp_year,
        planting_year,
        region,
        status
    FROM masterdatabase.contract_tree_information
    WHERE contract_code = ANY(:codes)
    ORDER BY etp_year, region, contract_code;
""")

df_33_status = pd.read_sql_query(
    status_33_query,
    engine,
    params={'codes': list(codes_33_in_source)}
)

print("\n" + "=" * 80)
print(f"📋 LOS {len(codes_33_in_source)} CONTRATOS QUE SÍ TIENEN DATOS SOURCE")
print("=" * 80)
print("\n⚠️  Estos contratos SÍ tienen datos en tablas inventory_* pero:")
print("   - No fueron procesados correctamente a IM")
print("   - O fueron procesados pero sin survival")
print("\n")
print(df_33_status.to_string(index=False))

# ============================================================================
# Guardar resultados
# ============================================================================
globals()['df_86_status'] = df_86_status
globals()['df_out_of_program'] = df_out_of_program
globals()['df_not_out_of_program'] = df_not_out_of_program
globals()['df_33_in_source'] = df_33_status

print("\n\n✅ DataFrames disponibles:")
print("   - df_86_status (todos los 86)")
print("   - df_out_of_program (los que SÍ son Out of Program)")
print("   - df_not_out_of_program (los que NO son Out of Program)")
print("   - df_33_in_source (los 33 que sí tienen datos source)")

💻 Conectado a la base de datos helloworldtree
🔍 ANÁLISIS DE LOS 30 CONTRATOS SIN DATOS SOURCE

📊 Distribución por STATUS:
--------------------------------------------------------------------------------
status_clean
Active       24
RFT           3
Rescinded     3

📊 Distribución por REGION:
--------------------------------------------------------------------------------
region
USA           22
Mexico         4
Costa Rica     3
Guatemala      1

📊 Distribución por ETP_YEAR:
--------------------------------------------------------------------------------
etp_year
2020    16
2018     6
2021     5
2022     2
2015     1

🚨 HALLAZGOS CRÍTICOS

 De los 30 contratos sin datos source:
   ✅ 0 son 'Out of Program' (ESPERADO)
   ⚠️  30 NO son 'Out of Program' (PROBLEMÁTICO)

🚨 Contratos SIN datos source que NO son 'Out of Program':
--------------------------------------------------------------------------------
contract_code  etp_year  planting_year     region    status status_clean
       US0001 

In [20]:
from core.db import get_engine
from core.libs import pd, text

engine = get_engine()

codes_33 = [
    'CR0037','US0090','US0094','US0116','CR0099','CR0101','CR0105',
    'MX0027','MX0028','MX0030','MX0033','MX0036','MX0037','US0110',
    'US0129','US0130','US0132','US0134','US0135','US0136','US0138',
    'US0142','US0143','US0144','US0145','US0150','US0154','US0156',
    'US0157','US0161','US0155','US0165','US0163'
]

# 1. Buscar tablas source
tables_query = text("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
      AND table_name LIKE 'inventory_%'
    ORDER BY table_name;
""")

df_tables = pd.read_sql_query(tables_query, engine)
inventory_tables = df_tables['table_name'].tolist()

results = []

for table in inventory_tables:
    q = text(f"""
        SELECT
            '{table}' AS source_table,
            *
        FROM public.{table}
        WHERE contractcode = ANY(:codes)
    """)

    try:
        df = pd.read_sql_query(q, engine, params={"codes": codes_33})
        if not df.empty:
            results.append(df)
    except:
        continue

if results:
    df_source_33 = pd.concat(results, ignore_index=True)
    print("✔ Source encontrado para los 33 contratos")
else:
    print("⚠ No se encontró información source")

globals()['df_source_33'] = df_source_33
df_source_33.head()


💻 Conectado a la base de datos helloworldtree
✔ Source encontrado para los 33 contratos


C:\Users\HeyCe\AppData\Local\Temp\ipykernel_4504\2098477322.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_source_33 = pd.concat(results, ignore_index=True)


,source_table,contractcode,farmername,cruisedate,id,id_error,stand,plot,plot_coordinate,tree_number,defect_ht_ft,dbh_in,tht_ft,merch_ht_ft,short_note,Species_id,Defect_id,Pests_id,Coppiced_id,Permanent Plot_id,Disease_id,doyle_bf,dead_tree,alive_tree,units_corrected,cruise_date
0,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001001,None,1.0,1,E520592 N1147686,1.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
1,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001002,None,1.0,1,E520592 N1147686,2.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
2,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001003,None,1.0,1,E520592 N1147686,3.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
3,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001004,None,1.0,1,E520592 N1147686,4.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
4,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001005,None,1.0,1,E520592 N1147686,5.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN


In [21]:
print(df_source_33['contractcode'].nunique())
df_source_33.head(33)

33


,source_table,contractcode,farmername,cruisedate,id,id_error,stand,plot,plot_coordinate,tree_number,defect_ht_ft,dbh_in,tht_ft,merch_ht_ft,short_note,Species_id,Defect_id,Pests_id,Coppiced_id,Permanent Plot_id,Disease_id,doyle_bf,dead_tree,alive_tree,units_corrected,cruise_date
0,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001001,None,1.0,1,E520592 N1147686,1.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
1,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001002,None,1.0,1,E520592 N1147686,2.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
2,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001003,None,1.0,1,E520592 N1147686,3.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
3,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001004,None,1.0,1,E520592 N1147686,4.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
4,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001005,None,1.0,1,E520592 N1147686,5.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
5,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001006,None,1.0,1,E520592 N1147686,6.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
6,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001007,None,1.0,1,E520592 N1147686,7.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
7,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001008,None,1.0,1,E520592 N1147686,8.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
8,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001009,None,1.0,1,E520592 N1147686,9.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN
9,inventory_cr_2024,CR0037,Hermanos Wolfe Arce S.A,2024-02-09 00:00:00,CR003701001010,None,1.0,1,E520592 N1147686,10.0,None,None,NaN,NaN,Termitas,None,None,None,None,None,None,None,0.0,1.0,True,NaN


In [22]:
from core.db import get_engine
from core.libs import pd, text, re

engine = get_engine()

print("🔍 Buscando tablas public.inventory_% ...")

# 1. Listar todas las tablas
tables_query = text("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
      AND table_name LIKE 'inventory_%'
    ORDER BY table_name;
""")
df_tables = pd.read_sql_query(tables_query, engine)
inventory_tables = df_tables['table_name'].tolist()

print(f"✔ Se encontraron {len(inventory_tables)} tablas.")
for t in inventory_tables:
    print("   -", t)

all_frames = []

# Regex para detectar año en nombre de tabla
year_regex = re.compile(r"(20[1-3][0-9])")   # 2010–2039

for table in inventory_tables:
    print(f"\n📥 Leyendo {table} ...")

    # detectar columnas
    cols_query = text("""
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name = :table
    """)
    df_cols = pd.read_sql_query(cols_query, engine, params={"table": table})
    cols = df_cols['column_name'].tolist()

    # asegurarnos que existe contractcode
    if "contractcode" not in cols:
        print(f"⚠ {table} no tiene contractcode — omitida")
        continue

    # columnas que tomaremos
    select_cols = ["contractcode"]

    # si la tabla ya tiene inventory_year, lo tomamos
    if "inventory_year" in cols:
        select_cols.append("inventory_year")
        table_has_inventory_year = True
    else:
        table_has_inventory_year = False

    if "survival" in cols:
        select_cols.append("survival")
    if "dbh" in cols:
        select_cols.append("dbh")
    if "mht" in cols:
        select_cols.append("mht")

    # leer tabla
    col_str = ", ".join(select_cols)
    q = text(f"""
        SELECT {col_str}, '{table}' AS source_table
        FROM public.{table}
    """)

    df = pd.read_sql_query(q, engine)

    # ===========================
    # EXTRAER inventory_year POR NOMBRE DE TABLA
    # ===========================
    if not table_has_inventory_year:
        match = year_regex.search(table)
        if match:
            extracted_year = int(match.group(1))
        else:
            extracted_year = None  # no se pudo determinar

        df["inventory_year"] = extracted_year

    all_frames.append(df)


# 3. Concatenar todas las tablas
df_all_inventory = pd.concat(all_frames, ignore_index=True)

print("\n✔ Concatenación completa.")
print(f"Total de filas: {len(df_all_inventory):,}")
print(f"Columnas: {df_all_inventory.columns.tolist()}")

# ============================
# 4. region_code desde contractcode
# ============================
def extract_region(code):
    m = re.match(r"([A-Z]{2})", code)
    return m.group(1) if m else None

df_all_inventory["region_code"] = df_all_inventory["contractcode"].apply(extract_region)

globals()["df_all_inventory"] = df_all_inventory

print("\n🏁 df_all_inventory creado exitosamente.")
df_all_inventory.head()


💻 Conectado a la base de datos helloworldtree
🔍 Buscando tablas public.inventory_% ...
✔ Se encontraron 16 tablas.
   - inventory_cr_2021
   - inventory_cr_2022
   - inventory_cr_2024
   - inventory_cr_2025
   - inventory_gt_2021
   - inventory_gt_2022
   - inventory_gt_2024
   - inventory_gt_2025
   - inventory_mx_2021
   - inventory_mx_2022
   - inventory_mx_2024
   - inventory_mx_2025
   - inventory_us_2021
   - inventory_us_2022
   - inventory_us_2024
   - inventory_us_2025

📥 Leyendo inventory_cr_2021 ...

📥 Leyendo inventory_cr_2022 ...

📥 Leyendo inventory_cr_2024 ...

📥 Leyendo inventory_cr_2025 ...

📥 Leyendo inventory_gt_2021 ...

📥 Leyendo inventory_gt_2022 ...

📥 Leyendo inventory_gt_2024 ...

📥 Leyendo inventory_gt_2025 ...

📥 Leyendo inventory_mx_2021 ...

📥 Leyendo inventory_mx_2022 ...

📥 Leyendo inventory_mx_2024 ...

📥 Leyendo inventory_mx_2025 ...

📥 Leyendo inventory_us_2021 ...

📥 Leyendo inventory_us_2022 ...

📥 Leyendo inventory_us_2024 ...

📥 Leyendo inventory_u

,contractcode,source_table,inventory_year,region_code
0,CR0081,inventory_cr_2021,2021,CR
1,CR0086,inventory_cr_2021,2021,CR
2,CR0086,inventory_cr_2021,2021,CR
3,CR0086,inventory_cr_2021,2021,CR
4,CR0086,inventory_cr_2021,2021,CR


In [23]:
print(len(df_all_inventory))
print(df_all_inventory.columns.tolist())
df_all_inventory.head()

55196
['contractcode', 'source_table', 'inventory_year', 'region_code']


,contractcode,source_table,inventory_year,region_code
0,CR0081,inventory_cr_2021,2021,CR
1,CR0086,inventory_cr_2021,2021,CR
2,CR0086,inventory_cr_2021,2021,CR
3,CR0086,inventory_cr_2021,2021,CR
4,CR0086,inventory_cr_2021,2021,CR


In [24]:
df_contract_inventory = df_all_inventory.drop_duplicates(
    subset=['contractcode', 'inventory_year']
)
df_contract_inventory.head()
print(len(df_contract_inventory))


379


In [25]:
df_IM = pd.read_sql_query("""
    SELECT DISTINCT contract_code, inventory_year
    FROM masterdatabase.inventory_metrics
""", engine)

print("Registros únicos en IM:", len(df_IM))
df_IM.head()

df_inv = df_contract_inventory.rename(columns={
    'contractcode': 'contract_code'
})

contracts_inv = set(df_inv['contract_code'])
contracts_IM = set(df_IM['contract_code'])

missing_in_IM = contracts_inv - contracts_IM
print("Contratos con inventario REAL pero SIN IM:", len(missing_in_IM))
sorted(list(missing_in_IM))[:20]  # primeros 20


ghost_IM = contracts_IM - contracts_inv
print("Contratos en IM que NO tienen inventario real:", len(ghost_IM))
sorted(list(ghost_IM))[:20]


Registros únicos en IM: 446
Contratos con inventario REAL pero SIN IM: 0
Contratos en IM que NO tienen inventario real: 182


['CR0001',
 'CR0002',
 'CR0003',
 'CR0004',
 'CR0005',
 'CR0006',
 'CR0007',
 'CR0008',
 'CR0009',
 'CR0010',
 'CR0011',
 'CR0012',
 'CR0014',
 'CR0015',
 'CR0016',
 'CR0017',
 'CR0018',
 'CR0019',
 'CR0020',
 'CR0021']

In [26]:
df_cti = pd.read_sql_query("""
    SELECT contract_code, status
    FROM masterdatabase.contract_tree_information
""", engine)

df_cti.head()


,contract_code,status
0,MX0071,Active
1,MX0099,Pending POD
2,US0164,Active
3,US0166,Active
4,US0168,Active


In [27]:
df_ghost_IM = pd.DataFrame(sorted(list(ghost_IM)), columns=['contract_code'])

df_ghost_IM = df_ghost_IM.merge(df_cti, on='contract_code', how='left')

df_ghost_IM.head()


,contract_code,status
0,CR0001,Out of Program
1,CR0002,Out of Program
2,CR0003,Out of Program
3,CR0004,Out of Program
4,CR0005,Out of Program


In [28]:
df_contract_inventory = df_all_inventory.drop_duplicates(
    subset=['contractcode', 'inventory_year']
)
df_contract_inventory.head()
print(len(df_contract_inventory))

df_simple = df_all_inventory[['contractcode', 'inventory_year']]

df_simple = df_simple.dropna(subset=['inventory_year'])

df_latest_inventory = (
    df_simple
    .groupby('contractcode', as_index=False)
    .agg({'inventory_year': 'max'})
)

print("Contratos únicos con inventario (más reciente):", len(df_latest_inventory))


379
Contratos únicos con inventario (más reciente): 242


In [29]:
df_null_surv = pd.read_sql_query("""
    SELECT contract_code
    FROM masterdatabase.inventory_metrics_current
    WHERE survival IS NULL
""", engine)

codes_null_surv = set(df_null_surv['contract_code'])
print("Contratos con survival NULL:", len(codes_null_surv))

df_ghost = pd.DataFrame(sorted(codes_null_surv), columns=['contract_code'])

df_cti = pd.read_sql_query("""
    SELECT contract_code, planting_year, etp_year, status, region
    FROM masterdatabase.contract_tree_information
""", engine)

df_ghost = df_ghost.merge(df_cti, on='contract_code', how='left')



Contratos con survival NULL: 159


In [30]:
df_pending = df_cti[df_cti['status']=='Pending POD']
print("Usando df_cti:")
print(len(df_pending))

df_pending = df_ghost[df_ghost['status']=='Pending POD']
print("Usando df_ghost:")
print(len(df_pending))

Usando df_cti:
34
Usando df_ghost:
0


### Relación CTI → IMC (contract_tree_information → inventory_metrics_current)

- Llave lógica: `contract_code`.
- CTI es la fuente de verdad de:
  - existencia del contrato
  - `status`
  - `planting_year`
- IMC es una tabla derivada que mantiene **un registro por contrato** con el inventario más reciente.

#### Reglas por `status` y edad (`contract_age = año_actual - planting_year`)

1. `status LIKE 'Pending%'`
   - No debe existir fila en IMC.
   - Si existe, se elimina (`DELETE FROM inventory_metrics_current`).

2. `status = 'Active'` y `contract_age < 3`
   - Debe existir fila en IMC.
   - `tree_age = contract_age`
   - `survival = 1.0` (100% por defecto hasta primer inventario).

3. `status = 'Active'` y `contract_age ≥ 3`
   - Debe existir fila en IMC.
   - `tree_age = contract_age`
   - `survival = NULL` (a la espera de datos reales de inventario).

4. `status = 'Rescinded'`
   - Puede existir fila en IMC para registro histórico.
   - `survival = NULL`.

5. `status = 'RFT'`
   - No se pisa `survival` a menos que venga un valor explícito.
   - Se actualizan edad y planting_year, pero se conserva el survival previo.

6. `status = 'Out of Program'`
   - No se incluye en el batch masivo.
   - El trigger está diseñado para no modificar survival (se preserva el valor histórico).


In [31]:
from core.db import get_engine
from core.libs import pd, text

engine = get_engine()

print("=" * 80)
print("🔍 EXAMEN DE RELACIONES: CTI → IM → IMC")
print("=" * 80)

# ============================================================================
# 1. CTI: Todos los contratos
# ============================================================================
print("\n📋 1. CONTRACT_TREE_INFORMATION (CTI) - Contratos por Status")
print("-" * 80)

q_cti = text("""
    SELECT
        status,
        COUNT(*) as n_contracts,
        MIN(planting_year) as earliest_planting,
        MAX(planting_year) as latest_planting
    FROM masterdatabase.contract_tree_information
    GROUP BY status
    ORDER BY n_contracts DESC;
""")

df_cti_summary = pd.read_sql_query(q_cti, engine)
print(df_cti_summary.to_string(index=False))

q_cti_total = text("""
    SELECT COUNT(*) as total_contracts
    FROM masterdatabase.contract_tree_information;
""")
df_cti_total = pd.read_sql_query(q_cti_total, engine)
print(f"\n📌 TOTAL CTI: {df_cti_total['total_contracts'].iloc[0]} contratos")

# ============================================================================
# 2. IMC: Contratos en inventory_metrics_current
# ============================================================================
print("\n\n📋 2. INVENTORY_METRICS_CURRENT (IMC)")
print("-" * 80)

q_imc = text("""
    SELECT
        COUNT(*) as total_rows,
        COUNT(DISTINCT contract_code) as unique_contracts,
        COUNT(survival) as rows_with_survival,
        COUNT(*) - COUNT(survival) as rows_null_survival
    FROM masterdatabase.inventory_metrics_current;
""")

df_imc_summary = pd.read_sql_query(q_imc, engine)
print(df_imc_summary.to_string(index=False))

# ============================================================================
# 3. Relación CTI → IMC
# ============================================================================
print("\n\n🔗 3. RELACIÓN CTI → IMC")
print("-" * 80)

q_relation = text("""
    SELECT
        cti.status,
        COUNT(DISTINCT cti.contract_code) as contracts_in_cti,
        COUNT(DISTINCT imc.contract_code) as contracts_in_imc,
        COUNT(DISTINCT cti.contract_code) - COUNT(DISTINCT imc.contract_code) as missing_in_imc
    FROM masterdatabase.contract_tree_information cti
    LEFT JOIN masterdatabase.inventory_metrics_current imc
        ON cti.contract_code = imc.contract_code
    GROUP BY cti.status
    ORDER BY contracts_in_cti DESC;
""")

df_relation = pd.read_sql_query(q_relation, engine)
print(df_relation.to_string(index=False))

# ============================================================================
# 4. ¿Qué contratos están en CTI pero NO en IMC?
# ============================================================================
print("\n\n❌ 4. CONTRATOS EN CTI PERO NO EN IMC")
print("-" * 80)

q_missing = text("""
    SELECT
        cti.contract_code,
        cti.status,
        cti.planting_year,
        cti.etp_year,
        cti.region,
        (2025 - cti.planting_year) as age
    FROM masterdatabase.contract_tree_information cti
    LEFT JOIN masterdatabase.inventory_metrics_current imc
        ON cti.contract_code = imc.contract_code
    WHERE imc.contract_code IS NULL
    ORDER BY cti.status, cti.planting_year DESC;
""")

df_missing_imc = pd.read_sql_query(q_missing, engine)
print(f"Total contratos faltantes en IMC: {len(df_missing_imc)}")
print("\nPrimeros 20:")
print(df_missing_imc.head(20).to_string(index=False))

print("\n📊 Por status:")
print(df_missing_imc['status'].value_counts().to_string())

# ============================================================================
# 5. ¿Cómo se crea IMC actualmente? (Views/Triggers)
# ============================================================================
print("\n\n🔧 5. DEFINICIÓN DE INVENTORY_METRICS_CURRENT")
print("-" * 80)

q_view_def = text("""
    SELECT
        table_type,
        is_insertable_into
    FROM information_schema.tables
    WHERE table_schema = 'masterdatabase'
      AND table_name = 'inventory_metrics_current';
""")

df_view_info = pd.read_sql_query(q_view_def, engine)
print(df_view_info.to_string(index=False))

# Intentar obtener la definición del view
try:
    q_view_source = text("""
        SELECT pg_get_viewdef('masterdatabase.inventory_metrics_current', true) as view_definition;
    """)
    df_view_source = pd.read_sql_query(q_view_source, engine)
    print("\n📜 Definición del VIEW/TABLE:")
    print(df_view_source['view_definition'].iloc[0])
except Exception as e:
    print(f"\n⚠️ No se pudo obtener definición: {e}")

# ============================================================================
# 6. Triggers en CTI o IMC
# ============================================================================
print("\n\n⚡ 6. TRIGGERS EXISTENTES")
print("-" * 80)

q_triggers = text("""
    SELECT
        trigger_name,
        event_manipulation,
        event_object_table,
        action_timing,
        action_statement
    FROM information_schema.triggers
    WHERE event_object_schema = 'masterdatabase'
      AND event_object_table IN ('contract_tree_information', 'inventory_metrics_current', 'inventory_metrics')
    ORDER BY event_object_table, trigger_name;
""")

df_triggers = pd.read_sql_query(q_triggers, engine)
if len(df_triggers) > 0:
    print(df_triggers.to_string(index=False))
else:
    print("❌ No hay triggers configurados en estas tablas")

# ============================================================================
# 7. VALIDACIÓN SEGÚN REGLAS DE NEGOCIO
# ============================================================================
print("\n\n" + "=" * 80)
print("🎯 VALIDACIÓN DE REGLAS DE NEGOCIO")
print("=" * 80)

q_validation = text("""
    SELECT
        cti.contract_code,
        cti.status,
        cti.planting_year,
        cti.etp_year,
        cti.region,
        (2025 - cti.planting_year) as age,
        imc.survival,
        imc.inventory_year,
        CASE
            -- Regla 1: Menor de 3 años
            WHEN (2025 - cti.planting_year) < 3 THEN
                CASE
                    WHEN cti.status = 'Active' AND imc.survival IS NULL
                        THEN '❌ ERROR: Active <3 años debe tener survival numérico'
                    WHEN cti.status = 'Active' AND imc.survival IS NOT NULL
                        THEN '✅ OK'
                    WHEN cti.status = 'Rescinded' AND imc.survival IS NOT NULL
                        THEN '⚠️ WARNING: Rescinded debería tener survival=NULL (contrato cancelado)'
                    WHEN cti.status = 'Rescinded' AND imc.survival IS NULL
                        THEN '✅ OK'
                    WHEN cti.status LIKE 'Pending%' AND imc.survival IS NOT NULL
                        THEN '⚠️ WARNING: Pending debería tener survival=NULL (no plantado)'
                    WHEN cti.status LIKE 'Pending%' AND imc.survival IS NULL
                        THEN '✅ OK'
                    ELSE '✅ OK'
                END
            -- Regla 2: 3 años o más
            WHEN (2025 - cti.planting_year) >= 3 THEN
                CASE
                    WHEN cti.status = 'Active' AND imc.survival IS NULL
                        THEN '❌ ERROR: Active ≥3 años debe tener survival numérico'
                    WHEN cti.status = 'Active' AND imc.survival IS NOT NULL
                        THEN '✅ OK'
                    WHEN cti.status = 'RFT' AND imc.survival IS NULL
                        THEN '⚠️ WARNING: RFT ≥3 años debería tener último valor conocido'
                    WHEN cti.status = 'Rescinded' AND imc.survival IS NOT NULL
                        THEN '⚠️ WARNING: Rescinded debería tener survival=NULL (contrato cancelado)'
                    WHEN cti.status = 'Rescinded' AND imc.survival IS NULL
                        THEN '✅ OK'
                    WHEN cti.status = 'Out of Program'
                        THEN '✅ OK (histórico)'
                    ELSE '✅ OK'
                END
            ELSE '❓ UNKNOWN'
        END as validation_status
    FROM masterdatabase.contract_tree_information cti
    LEFT JOIN masterdatabase.inventory_metrics_current imc
        ON cti.contract_code = imc.contract_code
    WHERE cti.status NOT IN ('Out of Program')
    ORDER BY validation_status, age DESC, cti.contract_code;
""")

df_validation = pd.read_sql_query(q_validation, engine)

# Contar por tipo de validación
print("\n📊 Resumen de Validación:")
print("-" * 80)
validation_summary = df_validation['validation_status'].value_counts()
print(validation_summary.to_string())

# Mostrar errores críticos
df_errors = df_validation[df_validation['validation_status'].str.contains('❌')]
if len(df_errors) > 0:
    print(f"\n\n❌ ERRORES CRÍTICOS ({len(df_errors)} contratos):")
    print("-" * 80)
    print(df_errors[['contract_code', 'status', 'age', 'survival', 'validation_status']].to_string(index=False))

# Mostrar warnings
df_warnings = df_validation[df_validation['validation_status'].str.contains('⚠️')]
if len(df_warnings) > 0:
    print(f"\n\n⚠️ ADVERTENCIAS ({len(df_warnings)} contratos):")
    print("-" * 80)
    print(df_warnings[['contract_code', 'status', 'age', 'survival', 'validation_status']].head(20).to_string(index=False))

# ============================================================================
# 8. Resumen de hallazgos
# ============================================================================
print("\n\n" + "=" * 80)
print("📊 RESUMEN DE HALLAZGOS")
print("=" * 80)

total_cti = df_cti_total['total_contracts'].iloc[0]
total_imc = df_imc_summary['unique_contracts'].iloc[0]
missing = len(df_missing_imc)
errors = len(df_errors) if len(df_errors) > 0 else 0
warnings = len(df_warnings) if len(df_warnings) > 0 else 0

print(f"""
✅ Total contratos en CTI: {total_cti}
✅ Total contratos en IMC: {total_imc}
❌ Faltantes en IMC: {missing}
❌ Errores de validación: {errors}
⚠️ Advertencias: {warnings}

📌 Contratos faltantes por status:
{df_missing_imc['status'].value_counts().to_string() if missing > 0 else "Ninguno"}

🎯 REGLAS DE NEGOCIO A APLICAR:

┌─────────────────────────────────────────────────────────────┐
│ SI EDAD < 3 AÑOS:                                           │
├─────────────────────────────────────────────────────────────┤
│  • Active    → survival = DEBE TENER VALOR numérico         │
│                Recomendado: 1.0 (100%) si no hay inventario │
│  • Rescinded → survival = NULL (contrato cancelado)         │
│  • Pending % → survival = NULL (aún no plantado)            │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│ SI EDAD ≥ 3 AÑOS:                                           │
├─────────────────────────────────────────────────────────────┤
│  • Active    → survival = valor de inventory_metrics        │
│                (si NULL = ERROR, necesita inventario)       │
│  • RFT       → mantener último valor conocido               │
│  • Rescinded → survival = NULL (contrato cancelado)         │
│  • Out of Program → mantener histórico                      │
└─────────────────────────────────────────────────────────────┘
""")

# Guardar resultados
globals()['df_cti_summary'] = df_cti_summary
globals()['df_imc_summary'] = df_imc_summary
globals()['df_relation'] = df_relation
globals()['df_missing_imc'] = df_missing_imc
globals()['df_triggers'] = df_triggers
globals()['df_validation'] = df_validation
if len(df_errors) > 0:
    globals()['df_errors'] = df_errors
if len(df_warnings) > 0:
    globals()['df_warnings'] = df_warnings

print("\n✅ DataFrames disponibles:")
print("   - df_cti_summary")
print("   - df_imc_summary")
print("   - df_relation")
print("   - df_missing_imc")
print("   - df_triggers")
print("   - df_validation (con reglas de negocio)")
if len(df_errors) > 0:
    print("   - df_errors (contratos con errores críticos)")
if len(df_warnings) > 0:
    print("   - df_warnings (contratos con advertencias)")

💻 Conectado a la base de datos helloworldtree
🔍 EXAMEN DE RELACIONES: CTI → IM → IMC

📋 1. CONTRACT_TREE_INFORMATION (CTI) - Contratos por Status
--------------------------------------------------------------------------------
        status  n_contracts  earliest_planting  latest_planting
        Active          269               2016             2025
Out of Program          139               2016             2022
   Pending POD           34               2025             2025
           RFT           17               2015             2022
     Rescinded            3               2021             2023

📌 TOTAL CTI: 462 contratos


📋 2. INVENTORY_METRICS_CURRENT (IMC)
--------------------------------------------------------------------------------
 total_rows  unique_contracts  rows_with_survival  rows_null_survival
        430               430                 271                 159


🔗 3. RELACIÓN CTI → IMC
---------------------------------------------------------------------------

In [3]:
from core.db import get_engine
from core.libs import pd, text, np

engine = get_engine()

💻 Conectado a la base de datos helloworldtree


In [4]:

# =====================================================================
# 1. Traer contratos con survival IS NULL después del join CTI → IMC
# =====================================================================
q_null_survival = text("""
    SELECT
        cti.contract_code,
        cti.status,
        cti.planting_year,
        cti.etp_year,
        cti.region,
        (2025 - cti.planting_year) AS age,
        imc.inventory_year,
        imc.survival
    FROM masterdatabase.contract_tree_information cti
    LEFT JOIN masterdatabase.inventory_metrics_current imc
        ON cti.contract_code = imc.contract_code
    WHERE imc.survival IS NULL;
""")

df_null = pd.read_sql_query(q_null_survival, engine)

print("Total contratos con survival NULL tras el join:", len(df_null))
print(df_null.head())

# =====================================================================
# 2. Clasificar en los 4 casos
# =====================================================================
def classify_null_case(row):
    # Caso 1: status = Out of Program
    if row["status"] == "Out of Program":
        return "CASE 1: Out of Program"
    # Caso 2: Edad < 3 años
    elif row["age"] < 3:
        return "CASE 2: Edad < 3 años"
    # Caso 3: Pending cases (cualquier variante de Pending%)
    elif isinstance(row["status"], str) and row["status"].startswith("Pending"):
        return "CASE 3: Pending cases"
    # Caso 4: todo lo demás
    else:
        return "CASE 4: Other"

df_null["null_survival_case"] = df_null.apply(classify_null_case, axis=1)

# =====================================================================
# 3. Resumen por caso
# =====================================================================
summary_cases = (
    df_null["null_survival_case"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "null_survival_case", "null_survival_case": "n_contracts"})
)

print("\n📊 Resumen por caso de survival NULL:")
print(summary_cases.to_string(index=False))

# Si quieres ver el detalle de cada caso:
df_case1 = df_null[df_null["null_survival_case"] == "CASE 1: Out of Program"]
df_case2 = df_null[df_null["null_survival_case"] == "CASE 2: Edad < 3 años"]
df_case3 = df_null[df_null["null_survival_case"] == "CASE 3: Pending cases"]
df_case4 = df_null[df_null["null_survival_case"] == "CASE 4: Other"]

print("\nCASE 4 (Other) - primeros 20:")
print(df_case4.head(20).to_string(index=False))


Total contratos con survival NULL tras el join: 244
  contract_code          status  planting_year  etp_year      region  age  \
0        CR0001  Out of Program           2016      2015  Costa Rica    9   
1        CR0002  Out of Program           2016      2015  Costa Rica    9   
2        CR0003  Out of Program           2016      2015  Costa Rica    9   
3        CR0004  Out of Program           2016      2015  Costa Rica    9   
4        CR0005  Out of Program           2016      2015  Costa Rica    9   

  inventory_year survival  
0           None     None  
1           None     None  
2           None     None  
3           None     None  
4           None     None  

📊 Resumen por caso de survival NULL:
           n_contracts  count
 CASE 2: Edad < 3 años    109
CASE 1: Out of Program    107
         CASE 4: Other     28

CASE 4 (Other) - primeros 20:
contract_code    status  planting_year  etp_year     region  age inventory_year survival null_survival_case
       CR0040    Act

In [35]:
# =====================================================================
# Verificar inventarios en tablas public.inventory_{country}_{year}
# =====================================================================

# Extraer códigos de país y años de los contratos CASE 4
df_case4['country_code'] = df_case4['contract_code'].str[:2]
df_case4['contractcode'] = df_case4['contract_code'].str.replace('_', '', regex=False)

# Obtener combinaciones únicas de país y año para buscar
# Buscaremos desde planting_year hasta 2024
years_to_check = list(range(2018, 2025))  # Ajusta según tus datos
countries = df_case4['country_code'].unique()

print(f"Países a revisar: {countries}")
print(f"Años a revisar: {years_to_check}")

# =====================================================================
# Función para verificar existencia de registros en inventarios
# =====================================================================
def check_inventory_tables(engine, df_case4):
    results = []

    for _, row in df_case4.iterrows():
        contract_code = row['contract_code']
        contractcode_no_underscore = row['contractcode']
        country = row['country_code']
        planting_year = row['planting_year']

        # Buscar desde el año de plantación hasta 2024
        for year in range(planting_year, 2025):
            table_name = f"public.inventory_{country.lower()}_{year}"

            # Verificar si la tabla existe y tiene el contrato
            query = text(f"""
                SELECT
                    '{contract_code}' as original_contract_code,
                    contractcode,
                    '{table_name}' as inventory_table,
                    COUNT(*) as row_count
                FROM {table_name}
                WHERE contractcode = :contractcode
                GROUP BY contractcode
            """)

            try:
                df_temp = pd.read_sql_query(
                    query,
                    engine,
                    params={'contractcode': contractcode_no_underscore}
                )

                if not df_temp.empty:
                    results.append({
                        'contract_code': contract_code,
                        'contractcode': contractcode_no_underscore,
                        'inventory_table': table_name,
                        'row_count': df_temp['row_count'].iloc[0],
                        'year': year
                    })

            except Exception as e:
                # La tabla no existe o hay error de permisos
                if "does not exist" not in str(e):
                    print(f"⚠️ Error en {table_name}: {e}")
                continue

    return pd.DataFrame(results)

# =====================================================================
# Ejecutar búsqueda
# =====================================================================
print("\n🔍 Buscando en tablas de inventario públicas...")
df_inventory_found = check_inventory_tables(engine, df_case4)

if not df_inventory_found.empty:
    print(f"\n✅ Se encontraron {len(df_inventory_found)} contratos con registros en inventarios:")
    print(df_inventory_found.to_string(index=False))

    # Resumen por tabla
    summary_by_table = df_inventory_found.groupby('inventory_table')['contract_code'].count()
    print("\n📊 Resumen por tabla de inventario:")
    print(summary_by_table)

    # Contratos que SÍ tienen inventario pero NO están en IMC
    contracts_with_inventory = df_inventory_found['contract_code'].unique()
    print(f"\n⚠️ Contratos CASE 4 que tienen inventario pero no están en IMC: {len(contracts_with_inventory)}")

else:
    print("\n❌ No se encontraron registros de inventario para los contratos CASE 4")

# =====================================================================
# Contratos sin ningún inventario
# =====================================================================
contracts_without_inventory = df_case4[
    ~df_case4['contract_code'].isin(df_inventory_found['contract_code'].unique())
]

print(f"\n📋 Contratos CASE 4 sin ningún registro de inventario: {len(contracts_without_inventory)}")
if not contracts_without_inventory.empty:
    print(contracts_without_inventory[['contract_code', 'status', 'planting_year', 'age', 'region']].to_string(index=False))

Países a revisar: ['MX' 'CR' 'US' 'GT']
Años a revisar: [2018, 2019, 2020, 2021, 2022, 2023, 2024]

🔍 Buscando en tablas de inventario públicas...


C:\Users\HeyCe\AppData\Local\Temp\ipykernel_4504\3215177737.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['country_code'] = df_case4['contract_code'].str[:2]
C:\Users\HeyCe\AppData\Local\Temp\ipykernel_4504\3215177737.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['contractcode'] = df_case4['contract_code'].str.replace('_', '', regex=False)



✅ Se encontraron 29 contratos con registros en inventarios:
contract_code contractcode          inventory_table  row_count  year
       CR0037       CR0037 public.inventory_cr_2024        259  2024
       US0116       US0116 public.inventory_us_2021         75  2021
       US0090       US0090 public.inventory_us_2021        122  2021
       US0094       US0094 public.inventory_us_2021        113  2021
       US0094       US0094 public.inventory_us_2022        156  2022
       US0094       US0094 public.inventory_us_2024         48  2024
       US0110       US0110 public.inventory_us_2021         62  2021
       US0110       US0110 public.inventory_us_2022         17  2022
       US0136       US0136 public.inventory_us_2022        107  2022
       US0129       US0129 public.inventory_us_2022         24  2022
       US0130       US0130 public.inventory_us_2022        109  2022
       US0135       US0135 public.inventory_us_2022         54  2022
       US0142       US0142 public.inventor

In [36]:
# =====================================================================
# Filtrar solo el año más reciente por contrato
# =====================================================================

# Obtener el año más reciente de inventario para cada contrato
df_inventory_latest = (
    df_inventory_found
    .sort_values('year', ascending=False)
    .groupby('contract_code')
    .first()
    .reset_index()
)

print(f"📊 Contratos con inventario (año más reciente): {len(df_inventory_latest)}")
print("\n" + df_inventory_latest.to_string(index=False))

# =====================================================================
# Verificar si estos contratos del año más reciente están en IMC
# =====================================================================

# Lista de contratos con su año más reciente
latest_contracts = df_inventory_latest['contract_code'].tolist()

# Consultar si estos están en IMC
q_check_imc = text("""
    SELECT
        contract_code,
        inventory_year,
        survival,
        trees_alive,
        trees_dead,
        last_updated
    FROM masterdatabase.inventory_metrics_current
    WHERE contract_code IN :contract_list
""")

df_in_imc = pd.read_sql_query(
    q_check_imc,
    engine,
    params={'contract_list': tuple(latest_contracts)}
)

print(f"\n🔍 Contratos que SÍ están en IMC: {len(df_in_imc)}")
if not df_in_imc.empty:
    print(df_in_imc.to_string(index=False))

# =====================================================================
# Identificar cuáles realmente faltan en IMC
# =====================================================================

contracts_truly_missing = df_inventory_latest[
    ~df_inventory_latest['contract_code'].isin(df_in_imc['contract_code'])
]

print(f"\n⚠️  CONTRATOS CON INVENTARIO RECIENTE PERO FALTANTES EN IMC: {len(contracts_truly_missing)}")
print("\n" + contracts_truly_missing.to_string(index=False))

# =====================================================================
# Comparar inventory_year entre la tabla pública y IMC
# =====================================================================

if not df_in_imc.empty:
    comparison = df_inventory_latest.merge(
        df_in_imc[['contract_code', 'inventory_year']],
        on='contract_code',
        how='left',
        suffixes=('_public', '_imc')
    )

    # Contratos donde el año público es más reciente que IMC
    outdated_imc = comparison[
        (comparison['inventory_year'].notna()) &
        (comparison['year'] > comparison['inventory_year'])
    ]

    if not outdated_imc.empty:
        print(f"\n📅 Contratos con IMC desactualizado (año público > año IMC): {len(outdated_imc)}")
        print("\n" + outdated_imc[['contract_code', 'year', 'inventory_year', 'inventory_table']].to_string(index=False))

# =====================================================================
# Actualizar lista de contratos sin inventario
# =====================================================================

# Remover de la lista "sin inventario" aquellos que sí tienen
contracts_with_recent_inv = df_inventory_latest['contract_code'].tolist()

contracts_without_inventory_updated = df_case4[
    ~df_case4['contract_code'].isin(contracts_with_recent_inv)
].copy()

print(f"\n📋 CONTRATOS SIN NINGÚN INVENTARIO (actualizado): {len(contracts_without_inventory_updated)}")
print("\n" + contracts_without_inventory_updated[['contract_code', 'status', 'planting_year', 'age', 'region']].to_string(index=False))

# =====================================================================
# Resumen final
# =====================================================================
print("\n" + "=" * 80)
print("📊 RESUMEN FINAL - CASE 4 (50 contratos)")
print("=" * 80)
print(f"\n1️⃣  Contratos con inventario reciente pero faltantes en IMC: {len(contracts_truly_missing)}")
print(f"2️⃣  Contratos con IMC desactualizado: {len(outdated_imc) if not df_in_imc.empty and not outdated_imc.empty else 0}")
print(f"3️⃣  Contratos sin ningún inventario registrado: {len(contracts_without_inventory_updated)}")
print(f"\n✅ Total verificado: {len(contracts_truly_missing) + (len(outdated_imc) if not df_in_imc.empty and not outdated_imc.empty else 0) + len(contracts_without_inventory_updated)} contratos")

📊 Contratos con inventario (año más reciente): 22

contract_code contractcode          inventory_table  row_count  year
       CR0037       CR0037 public.inventory_cr_2024        259  2024
       CR0099       CR0099 public.inventory_cr_2024        289  2024
       MX0027       MX0027 public.inventory_mx_2024        260  2024
       MX0028       MX0028 public.inventory_mx_2024        264  2024
       MX0030       MX0030 public.inventory_mx_2024        130  2024
       MX0033       MX0033 public.inventory_mx_2024        461  2024
       MX0036       MX0036 public.inventory_mx_2024        256  2024
       MX0037       MX0037 public.inventory_mx_2024         92  2024
       US0090       US0090 public.inventory_us_2021        122  2021
       US0094       US0094 public.inventory_us_2024         48  2024
       US0110       US0110 public.inventory_us_2022         17  2022
       US0116       US0116 public.inventory_us_2021         75  2021
       US0129       US0129 public.inventory_us_2022 

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "trees_alive" does not exist
LINE 6:         trees_alive,
                ^

[SQL: 
    SELECT
        contract_code,
        inventory_year,
        survival,
        trees_alive,
        trees_dead,
        last_updated
    FROM masterdatabase.inventory_metrics_current
    WHERE contract_code IN %(contract_list)s
]
[parameters: {'contract_list': ('CR0037', 'CR0099', 'MX0027', 'MX0028', 'MX0030', 'MX0033', 'MX0036', 'MX0037', 'US0090', 'US0094', 'US0110', 'US0116', 'US0129', 'US0130', 'US0132', 'US0134', 'US0135', 'US0136', 'US0138', 'US0142', 'US0143', 'US0144')}]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [37]:
# =====================================================================
# PATCH: Insertar 22 contratos faltantes usando el módulo InventoryMetrics
# =====================================================================

from InventoryMetrics.processing_metrics import aggregate_contracts
from InventoryMetrics.planting_times import pretty_tree_age
from InventoryMetrics.generate_helpers import clean_and_fuse_metrics
from InventoryMetrics.current_generator import regenerate_inventory_metrics_current
from core.libs import pd, text, re
from core.db import get_engine

engine = get_engine()

print("=" * 80)
print("🔧 INSERTANDO 22 CONTRATOS FALTANTES EN INVENTORY_METRICS")
print("=" * 80)

# =====================================================================
# Procesar cada tabla de inventario donde están los 22 contratos
# =====================================================================

# Agrupar por tabla de inventario
tables_to_process = df_inventory_latest.groupby('inventory_table').agg({
    'contract_code': list,
    'contractcode': list,
    'year': 'first'
}).reset_index()

print(f"\n📋 Tablas a procesar: {len(tables_to_process)}")

all_dfs = []

for _, table_info in tables_to_process.iterrows():
    table_name = table_info['inventory_table']
    contracts_in_table = table_info['contractcode']
    year = table_info['year']

    # Extraer country del nombre de la tabla: public.inventory_{country}_{year}
    m = re.match(r"public\.inventory_([a-z]+)_(\d{4})", table_name)
    if not m:
        print(f"⚠️  No se pudo parsear {table_name}")
        continue

    country, _ = m.groups()

    print(f"\n🔍 Procesando {table_name} ({len(contracts_in_table)} contratos)")

    # =====================================================================
    # Leer datos de la tabla pública filtrando solo los contratos faltantes
    # =====================================================================
    contracts_str = "', '".join(contracts_in_table)
    where_clause = f"contractcode IN ('{contracts_str}')"

    query = f"SELECT * FROM {table_name} WHERE {where_clause}"
    df_inv = pd.read_sql(query, engine)

    if df_inv.empty:
        print(f"   ⚠️  Sin datos")
        continue

    print(f"   📊 {len(df_inv)} registros encontrados")

    # =====================================================================
    # Usar aggregate_contracts() del módulo existente
    # =====================================================================
    df_metrics = aggregate_contracts(
        df_inv,
        engine,
        country=country,
        year=year,
        include_all_contracts=False,
        filter_valid=False  # No filtrar, queremos todos los datos disponibles
    )

    print(f"   ✅ {len(df_metrics)} contratos procesados")
    all_dfs.append(df_metrics)


💻 Conectado a la base de datos helloworldtree
🔧 INSERTANDO 22 CONTRATOS FALTANTES EN INVENTORY_METRICS

📋 Tablas a procesar: 5

🔍 Procesando public.inventory_cr_2024 (2 contratos)
   📊 548 registros encontrados
   ✅ 46 contratos procesados

🔍 Procesando public.inventory_mx_2024 (6 contratos)
   📊 1463 registros encontrados
   ✅ 138 contratos procesados

🔍 Procesando public.inventory_us_2021 (2 contratos)
   📊 197 registros encontrados
   ✅ 38 contratos procesados

🔍 Procesando public.inventory_us_2022 (7 contratos)
   📊 504 registros encontrados
   ✅ 7 contratos procesados

🔍 Procesando public.inventory_us_2024 (5 contratos)
   📊 454 registros encontrados
   ✅ 115 contratos procesados


In [38]:
# =====================================================================
# PATCH: Insertar 22 contratos faltantes usando el módulo InventoryMetrics
# =====================================================================

from InventoryMetrics.processing_metrics import aggregate_contracts
from InventoryMetrics.planting_times import pretty_tree_age
from InventoryMetrics.generate_helpers import clean_and_fuse_metrics
from InventoryMetrics.current_generator import regenerate_inventory_metrics_current
from core.libs import pd, text, re
from core.db import get_engine

engine = get_engine()

print("=" * 80)
print("🔧 INSERTANDO 22 CONTRATOS FALTANTES EN INVENTORY_METRICS")
print("=" * 80)

# =====================================================================
# Procesar cada tabla de inventario donde están los 22 contratos
# =====================================================================

# Agrupar por tabla de inventario
tables_to_process = df_inventory_latest.groupby('inventory_table').agg({
    'contract_code': list,
    'contractcode': list,
    'year': 'first'
}).reset_index()

print(f"\n📋 Tablas a procesar: {len(tables_to_process)}")

all_dfs = []

for _, table_info in tables_to_process.iterrows():
    table_name = table_info['inventory_table']
    contracts_in_table = table_info['contractcode']
    year = table_info['year']

    # Extraer country del nombre de la tabla: public.inventory_{country}_{year}
    m = re.match(r"public\.inventory_([a-z]+)_(\d{4})", table_name)
    if not m:
        print(f"⚠️  No se pudo parsear {table_name}")
        continue

    country, _ = m.groups()

    print(f"\n🔍 Procesando {table_name} ({len(contracts_in_table)} contratos)")

    # =====================================================================
    # Leer datos de la tabla pública filtrando solo los contratos faltantes
    # =====================================================================
    contracts_str = "', '".join(contracts_in_table)
    where_clause = f"contractcode IN ('{contracts_str}')"

    query = f"SELECT * FROM {table_name} WHERE {where_clause}"
    df_inv = pd.read_sql(query, engine)

    if df_inv.empty:
        print(f"   ⚠️  Sin datos")
        continue

    print(f"   📊 {len(df_inv)} registros encontrados")

    # =====================================================================
    # Usar aggregate_contracts() del módulo existente
    # =====================================================================
    df_metrics = aggregate_contracts(
        df_inv,
        engine,
        country=country,
        year=year,
        include_all_contracts=False,
        filter_valid=False  # No filtrar, queremos todos los datos disponibles
    )

    print(f"   ✅ {len(df_metrics)} contratos procesados")
    all_dfs.append(df_metrics)

# =====================================================================
# Consolidar todos los DataFrames
# =====================================================================
if all_dfs:
    df_new_metrics = pd.concat(all_dfs, ignore_index=True)
    print(f"\n📊 Total de registros generados: {len(df_new_metrics)}")

    # =====================================================================
    # CRÍTICO: Convertir survival y mortality de string a float
    # =====================================================================
    def clean_percentage(value):
        """Convierte '49.03%' a 49.03 (float)"""
        if pd.isna(value):
            return None
        if isinstance(value, str):
            return float(value.replace('%', ''))
        return float(value)

    df_new_metrics['survival'] = df_new_metrics['survival'].apply(clean_percentage)
    df_new_metrics['mortality'] = df_new_metrics['mortality'].apply(clean_percentage)

    print("✅ Porcentajes convertidos a float")

    # =====================================================================
    # Usar clean_and_fuse_metrics() para normalizar
    # =====================================================================
    df_new_metrics = clean_and_fuse_metrics(df_new_metrics)

    # =====================================================================
    # Agregar planting info y calcular tree_age
    # =====================================================================
    contracts_info = pd.read_sql(
        "SELECT contract_code, planting_year, planting_date FROM masterdatabase.contract_tree_information",
        engine
    )

    df_new_metrics = df_new_metrics.merge(
        contracts_info,
        on="contract_code",
        how="left"
    )

    # Calcular tree_age usando la función existente
    df_new_metrics["tree_age"] = [
        pretty_tree_age(row["planting_date"], row["inventory_date"])
        if pd.notna(row["planting_date"]) and pd.notna(row["inventory_date"])
        else None
        for _, row in df_new_metrics.iterrows()
    ]

    # Limpiar columnas duplicadas (por el merge)
    for col in ['planting_year', 'planting_date']:
        y_col = f"{col}_y"
        x_col = f"{col}_x"
        if y_col in df_new_metrics.columns:
            df_new_metrics[col] = df_new_metrics[y_col]
        elif x_col in df_new_metrics.columns:
            df_new_metrics[col] = df_new_metrics[x_col]
        for suffix in [y_col, x_col]:
            if suffix in df_new_metrics.columns:
                df_new_metrics.drop(suffix, axis=1, inplace=True)

    # Agregar type_of_metric
    if "type_of_metric" not in df_new_metrics.columns:
        df_new_metrics["type_of_metric"] = "inventory"

    # =====================================================================
    # Verificar tipos de datos antes de insertar
    # =====================================================================
    print("\n🔍 Verificando tipos de datos:")
    print(f"   survival: {df_new_metrics['survival'].dtype}")
    print(f"   mortality: {df_new_metrics['mortality'].dtype}")
    print(f"   Ejemplo survival: {df_new_metrics['survival'].iloc[0]}")
    print(f"   Ejemplo mortality: {df_new_metrics['mortality'].iloc[0]}")

    # =====================================================================
    # Insertar en inventory_metrics
    # =====================================================================
    print(f"\n💾 Insertando {len(df_new_metrics)} registros en inventory_metrics...")

    df_new_metrics.to_sql(
        "inventory_metrics",
        engine,
        schema="masterdatabase",
        if_exists="append",
        index=False
    )

    print("✅ Registros insertados")

    # =====================================================================
    # Regenerar inventory_metrics_current usando la función del módulo
    # =====================================================================
    print("\n🔄 Regenerando inventory_metrics_current...")
    regenerate_inventory_metrics_current(engine)

    # =====================================================================
    # Verificar que ahora estén en IMC
    # =====================================================================
    print("\n🔍 Verificando que los contratos estén en inventory_metrics_current...")

    contracts_list = df_new_metrics['contract_code'].unique().tolist()
    df_verify = pd.read_sql(
        text("""
            SELECT contract_code, inventory_year, survival, total_trees
            FROM masterdatabase.inventory_metrics_current
            WHERE contract_code IN :contracts
            AND survival IS NOT NULL
        """),
        engine,
        params={'contracts': tuple(contracts_list)}
    )

    print(f"\n✅ Contratos encontrados en IMC con survival: {len(df_verify)}/{len(contracts_list)}")
    if not df_verify.empty:
        print("\n" + df_verify.to_string(index=False))

    print("\n" + "=" * 80)
    print("✅ PROCESO COMPLETADO")
    print("=" * 80)

else:
    print("\n❌ No se generaron registros para insertar")

💻 Conectado a la base de datos helloworldtree
🔧 INSERTANDO 22 CONTRATOS FALTANTES EN INVENTORY_METRICS

📋 Tablas a procesar: 5

🔍 Procesando public.inventory_cr_2024 (2 contratos)
   📊 548 registros encontrados
   ✅ 46 contratos procesados

🔍 Procesando public.inventory_mx_2024 (6 contratos)
   📊 1463 registros encontrados
   ✅ 138 contratos procesados

🔍 Procesando public.inventory_us_2021 (2 contratos)
   📊 197 registros encontrados
   ✅ 38 contratos procesados

🔍 Procesando public.inventory_us_2022 (7 contratos)
   📊 504 registros encontrados
   ✅ 7 contratos procesados

🔍 Procesando public.inventory_us_2024 (5 contratos)
   📊 454 registros encontrados
   ✅ 115 contratos procesados

📊 Total de registros generados: 344
✅ Porcentajes convertidos a float

🔍 Verificando tipos de datos:
   survival: float64
   mortality: float64
   Ejemplo survival: 49.03
   Ejemplo mortality: 50.97

💾 Insertando 22 registros en inventory_metrics...
✅ Registros insertados

🔄 Regenerando inventory_metrics

In [40]:
df_verify = pd.read_sql(
    text("""
        SELECT contract_code, inventory_year, survival, total_trees
        FROM masterdatabase.inventory_metrics_current
        WHERE contract_code IN :contracts
        AND survival IS NOT NULL
        AND rn = 1  -- ⬅️ AGREGAR ESTO
    """),
    engine,
    params={'contracts': tuple(contracts_list)}
)

print (f"\n✅ Contratos encontrados en IMC con survival: {len(df_verify)}/{len(contracts_list)}")


✅ Contratos encontrados en IMC con survival: 0/22


In [41]:
# =====================================================================
# DIAGNÓSTICO: Por qué los contratos no aparecen con rn=1
# =====================================================================

from core.libs import pd, text
from core.db import get_engine

engine = get_engine()

print("=" * 80)
print("🔍 DIAGNÓSTICO: Contratos en IMC")
print("=" * 80)

# Lista de los 22 contratos
contracts_list = [
    'CR0037', 'CR0099', 'MX0027', 'MX0028', 'MX0030', 'MX0033', 'MX0036', 'MX0037',
    'US0090', 'US0094', 'US0110', 'US0116', 'US0129', 'US0130', 'US0132', 'US0134',
    'US0135', 'US0136', 'US0138', 'US0142', 'US0143', 'US0144'
]

# 1. Ver TODOS los registros de estos contratos en IMC (sin filtro rn)
print("\n1️⃣ Verificando registros en IMC (sin filtro rn):")
df_all = pd.read_sql(
    text("""
        SELECT contract_code, inventory_year, survival, rn
        FROM masterdatabase.inventory_metrics_current
        WHERE contract_code IN :contracts
        ORDER BY contract_code, rn
    """),
    engine,
    params={'contracts': tuple(contracts_list)}
)

print(f"   Total registros encontrados: {len(df_all)}")
if not df_all.empty:
    print("\n" + df_all.head(30).to_string(index=False))
    print(f"\n   Valores únicos de rn: {df_all['rn'].unique()}")
else:
    print("   ❌ No se encontraron registros en IMC")

# 2. Ver qué hay en inventory_metrics para estos contratos
print("\n2️⃣ Verificando registros en inventory_metrics:")
df_im = pd.read_sql(
    text("""
        SELECT contract_code, inventory_year, survival, pkid
        FROM masterdatabase.inventory_metrics
        WHERE contract_code IN :contracts
        ORDER BY contract_code, inventory_year DESC
    """),
    engine,
    params={'contracts': tuple(contracts_list)}
)

print(f"   Total registros en IM: {len(df_im)}")
if not df_im.empty:
    print("\n" + df_im.head(30).to_string(index=False))
else:
    print("   ❌ No se encontraron registros en IM")

# 3. Comparar: ¿Los 22 están en IM pero no en IMC?
contracts_in_im = set(df_im['contract_code'].unique()) if not df_im.empty else set()
contracts_in_imc = set(df_all['contract_code'].unique()) if not df_all.empty else set()

missing_in_imc = contracts_in_im - contracts_in_imc

if missing_in_imc:
    print(f"\n3️⃣ ⚠️ Contratos en IM pero NO en IMC: {len(missing_in_imc)}")
    print(f"   {sorted(missing_in_imc)}")
else:
    print("\n3️⃣ ✅ Todos los contratos de IM están en IMC")

# 4. Ver un ejemplo específico de la query de regeneración
print("\n4️⃣ Simulando la query de IMC para un contrato:")
sample_contract = contracts_list[0]
df_sample = pd.read_sql(
    text("""
        SELECT
            im.contract_code,
            im.inventory_year,
            im.survival,
            im.pkid,
            cti.status AS contract_status,
            ROW_NUMBER() OVER (PARTITION BY im.contract_code ORDER BY im.inventory_year DESC) as rn
        FROM masterdatabase.inventory_metrics im
        LEFT JOIN masterdatabase.contract_tree_information cti
            ON im.contract_code = cti.contract_code
        WHERE im.contract_code = :cc
    """),
    engine,
    params={'cc': sample_contract}
)

print(f"\n   Ejemplo para {sample_contract}:")
if not df_sample.empty:
    print(df_sample.to_string(index=False))
else:
    print(f"   ❌ No se encontró {sample_contract} en IM")

print("\n" + "=" * 80)

💻 Conectado a la base de datos helloworldtree
🔍 DIAGNÓSTICO: Contratos en IMC

1️⃣ Verificando registros en IMC (sin filtro rn):
   Total registros encontrados: 132

contract_code  inventory_year  survival  rn
       CR0037             NaN       NaN   1
       CR0037          2024.0     49.03   2
       CR0037          2024.0     49.03   3
       CR0037          2024.0     49.03   4
       CR0037          2024.0     49.03   5
       CR0037          2024.0     49.03   6
       CR0099             NaN       NaN   1
       CR0099          2024.0     89.27   2
       CR0099          2024.0     89.27   3
       CR0099          2024.0     89.27   4
       CR0099          2024.0     89.27   5
       CR0099          2024.0     89.27   6
       MX0027             NaN       NaN   1
       MX0027          2024.0     97.69   2
       MX0027          2024.0     97.69   3
       MX0027          2024.0     97.69   4
       MX0027          2024.0     97.69   5
       MX0027          2024.0     97.69   

In [44]:
# =====================================================================
# SOLUCIÓN: Deshabilitar trigger, limpiar duplicados, rehabilitar
# =====================================================================

from core.libs import pd, text
from core.db import get_engine

engine = get_engine()

print("=" * 80)
print("🧹 LIMPIANDO DUPLICADOS (CON TRIGGER DISABLED)")
print("=" * 80)

contracts_list = [
    'CR0037', 'CR0099', 'MX0027', 'MX0028', 'MX0030', 'MX0033', 'MX0036', 'MX0037',
    'US0090', 'US0094', 'US0110', 'US0116', 'US0129', 'US0130', 'US0132', 'US0134',
    'US0135', 'US0136', 'US0138', 'US0142', 'US0143', 'US0144'
]

with engine.begin() as conn:
    # 1. Deshabilitar el trigger
    print("\n1️⃣ Deshabilitando trigger sync_im_to_imc...")
    conn.execute(text("""
        ALTER TABLE masterdatabase.inventory_metrics
        DISABLE TRIGGER ALL
    """))
    print("   ✅ Trigger deshabilitado")

    # 2. Eliminar registros NULL
    print("\n2️⃣ Eliminando registros con survival=NULL...")
    result = conn.execute(
        text("""
            DELETE FROM masterdatabase.inventory_metrics
            WHERE contract_code IN :contracts
            AND survival IS NULL
            AND inventory_year IS NULL
        """),
        {'contracts': tuple(contracts_list)}
    )
    deleted_null = result.rowcount
    print(f"   ✅ Eliminados {deleted_null} registros NULL")

    # 3. Eliminar duplicados manteniendo solo uno
    print("\n3️⃣ Eliminando duplicados...")
    result = conn.execute(
        text("""
            DELETE FROM masterdatabase.inventory_metrics
            WHERE ctid NOT IN (
                SELECT MIN(ctid)
                FROM masterdatabase.inventory_metrics
                WHERE contract_code IN :contracts
                  AND survival IS NOT NULL
                GROUP BY contract_code, inventory_year
            )
            AND contract_code IN :contracts
            AND survival IS NOT NULL
        """),
        {'contracts': tuple(contracts_list)}
    )
    deleted_dupes = result.rowcount
    print(f"   ✅ Eliminados {deleted_dupes} registros duplicados")

    # 4. Rehabilitar el trigger
    print("\n4️⃣ Rehabilitando trigger...")
    conn.execute(text("""
        ALTER TABLE masterdatabase.inventory_metrics
        ENABLE TRIGGER ALL
    """))
    print("   ✅ Trigger rehabilitado")

# =====================================================================
# Verificar limpieza
# =====================================================================
print("\n5️⃣ Verificando limpieza en inventory_metrics:")
df_after = pd.read_sql(
    text("""
        SELECT contract_code, inventory_year, survival, pkid
        FROM masterdatabase.inventory_metrics
        WHERE contract_code IN :contracts
        ORDER BY contract_code, inventory_year DESC
    """),
    engine,
    params={'contracts': tuple(contracts_list)}
)

print(f"   Total registros: {len(df_after)}")
print("\n" + df_after.to_string(index=False))

# Verificar duplicados
dupes = df_after.groupby(['contract_code', 'inventory_year']).size()
dupes = dupes[dupes > 1]

if dupes.empty:
    print("\n   ✅ No hay duplicados en IM")
else:
    print(f"\n   ⚠️ Todavía hay {len(dupes)} duplicados")
    print(dupes)

# =====================================================================
# REGENERAR inventory_metrics_current
# =====================================================================
print("\n6️⃣ Regenerando inventory_metrics_current...")

from InventoryMetrics.current_generator import regenerate_inventory_metrics_current
regenerate_inventory_metrics_current(engine)

# =====================================================================
# VERIFICACIÓN FINAL
# =====================================================================
print("\n7️⃣ Verificación final en IMC:")

df_final = pd.read_sql(
    text("""
        SELECT contract_code, inventory_year, survival, total_trees, rn
        FROM masterdatabase.inventory_metrics_current
        WHERE contract_code IN :contracts
        AND rn = 1
        ORDER BY contract_code
    """),
    engine,
    params={'contracts': tuple(contracts_list)}
)

print(f"\n✅ Contratos en IMC con rn=1 y survival: {len(df_final)}/{len(contracts_list)}")

if not df_final.empty:
    # Contar cuántos tienen survival NOT NULL
    with_survival = df_final[df_final['survival'].notna()]
    print(f"   • Con survival válido: {len(with_survival)}")
    print("\n" + df_final.to_string(index=False))
else:
    print("   ❌ No se encontraron registros")

print("\n" + "=" * 80)
print("✅ LIMPIEZA COMPLETADA")
print("=" * 80)

💻 Conectado a la base de datos helloworldtree
🧹 LIMPIANDO DUPLICADOS (CON TRIGGER DISABLED)

1️⃣ Deshabilitando trigger sync_im_to_imc...
   ✅ Trigger deshabilitado

2️⃣ Eliminando registros con survival=NULL...
   ✅ Eliminados 22 registros NULL

3️⃣ Eliminando duplicados...
   ✅ Eliminados 88 registros duplicados

4️⃣ Rehabilitando trigger...
   ✅ Trigger rehabilitado

5️⃣ Verificando limpieza en inventory_metrics:
   Total registros: 22

contract_code  inventory_year  survival        pkid
       CR0037            2024     49.03 CR0037 2024
       CR0099            2024     89.27 CR0099 2024
       MX0027            2024     97.69 MX0027 2024
       MX0028            2024     96.21 MX0028 2024
       MX0030            2024     24.62 MX0030 2024
       MX0033            2024     98.26 MX0033 2024
       MX0036            2024     66.02 MX0036 2024
       MX0037            2024     11.96 MX0037 2024
       US0090            2021     73.77 US0090 2021
       US0094            2024     31

In [5]:
# =====================================================================
# UPDATE: Asignar survival=100 a CASE 2 (edad < 3, status=Active) en IMC
# =====================================================================

from core.libs import pd, text
from core.db import get_engine

engine = get_engine()

print("=" * 80)
print("🌱 ACTUALIZANDO SURVIVAL=100% PARA CASE 2 (EDAD < 3, STATUS=ACTIVE)")
print("=" * 80)

# =====================================================================
# 1. Identificar contratos CASE 2 con status=Active y survival=NULL
# =====================================================================
print("\n1️⃣ Identificando contratos CASE 2 con status=Active y survival=NULL...")

df_case2_active = pd.read_sql(
    text("""
        SELECT
            imc.contract_code,
            cti.status,
            cti.planting_year,
            (2025 - cti.planting_year) AS age,
            imc.survival,
            imc.rn
        FROM masterdatabase.inventory_metrics_current imc
        JOIN masterdatabase.contract_tree_information cti
            ON imc.contract_code = cti.contract_code
        WHERE imc.survival IS NULL
            AND (2025 - cti.planting_year) < 3
            AND cti.status = 'Active'
            AND imc.rn = 1
    """),
    engine
)

print(f"   Total contratos encontrados: {len(df_case2_active)}")

if df_case2_active.empty:
    print("\n   ℹ️  No hay contratos CASE 2 con status=Active sin survival")
    print("\n" + "=" * 80)
else:
    print("\n" + df_case2_active.to_string(index=False))

    # =====================================================================
    # 2. UPDATE en inventory_metrics_current
    # =====================================================================
    print("\n2️⃣ Actualizando survival=100, mortality=0 en IMC...")

    contracts_to_update = df_case2_active['contract_code'].tolist()

    with engine.begin() as conn:
        result = conn.execute(
            text("""
                UPDATE masterdatabase.inventory_metrics_current
                SET
                    survival = 100.0,
                    mortality = 0.0
                WHERE contract_code IN :contracts
                    AND rn = 1
            """),
            {'contracts': tuple(contracts_to_update)}
        )

        updated_count = result.rowcount
        print(f"   ✅ {updated_count} registros actualizados")

    # =====================================================================
    # 3. Verificar resultado
    # =====================================================================
    print("\n3️⃣ Verificación:")

    df_verify = pd.read_sql(
        text("""
            SELECT
                imc.contract_code,
                imc.inventory_year,
                imc.survival,
                imc.mortality,
                cti.status,
                (2025 - cti.planting_year) AS age
            FROM masterdatabase.inventory_metrics_current imc
            JOIN masterdatabase.contract_tree_information cti
                ON imc.contract_code = cti.contract_code
            WHERE imc.contract_code IN :contracts
                AND imc.rn = 1
            ORDER BY imc.contract_code
        """),
        engine,
        params={'contracts': tuple(contracts_to_update)}
    )

    print(f"\n✅ Contratos actualizados: {len(df_verify)}")

    if not df_verify.empty:
        print("\n" + df_verify.to_string(index=False))

        # Verificar que todos tengan survival=100
        with_100 = df_verify[df_verify['survival'] == 100.0]
        print(f"\n   • Con survival = 100%: {len(with_100)}/{len(df_verify)}")

    print("\n" + "=" * 80)
    print("✅ ACTUALIZACIÓN COMPLETADA")
    print("=" * 80)

💻 Conectado a la base de datos helloworldtree
🌱 ACTUALIZANDO SURVIVAL=100% PARA CASE 2 (EDAD < 3, STATUS=ACTIVE)

1️⃣ Identificando contratos CASE 2 con status=Active y survival=NULL...
   Total contratos encontrados: 73

contract_code status  planting_year  age survival  rn
       CR0101 Active           2023    2     None   1
       CR0103 Active           2023    2     None   1
       CR0105 Active           2023    2     None   1
       CR0106 Active           2024    1     None   1
       CR0108 Active           2024    1     None   1
       CR0109 Active           2024    1     None   1
       CR0110 Active           2024    1     None   1
       CR0111 Active           2024    1     None   1
      GT0031c Active           2023    2     None   1
       MX0050 Active           2023    2     None   1
       MX0053 Active           2024    1     None   1
       MX0054 Active           2024    1     None   1
       MX0055 Active           2024    1     None   1
       MX0056 Active  